<font size="5">**Major lines for all sources**

Given a .fits file, this code plots the most important emission and absorption lines of the spectrum of the source. The baseline around the line is considered the continuum level, used to normalize the line by dividing the line profile by it.

In [ ]:
import glob
import os
import warnings
from astropy.io import fits
from astropy import units as u
import numpy as np
from matplotlib import pyplot as plt
from specutils import Spectrum1D
from specutils.fitting.continuum import fit_continuum
from specutils import SpectralRegion
from astropy.visualization import quantity_support
from astropy.constants import c
quantity_support() 

c=c.to('km/s')

#Funtion to get the continuum level and divide the line between it
def fit_continuum(wl, flux, center, amplitude, vmax):
    wlmax=np.round((vmax*center/c)+center)
    delta_lambda=wlmax-center
    wlmin=center-delta_lambda
    mask=(wl>wlmin) & (wl<wlmax)
    wl_amplitude=(amplitude*center/c)+center
    
    mask_continuum=((wl>wlmin) & (wl<np.round(0.7*(center-wlmin)))) | ((wl>center+np.round(0.3*(wlmax-center))) & (wl<wlmax))
    mean=np.mean(flux[mask_continuum])
    norm_flux=flux[mask]/mean

    spec=Spectrum1D(spectral_axis=wl[mask], flux=norm_flux* u.Unit('erg cm-2 s-1 AA-1'))
    spec=degrade_resolution(spec, 0.2)
    v=c*(spec.spectral_axis[20:-20]-center)/center
    norm_flux=spec.flux[20:-20]
    
    return v, norm_flux


    
def spectrum_major_lines(file):
    f=fits.open(file)
    wl, flux=read_fits_spectrum(file)
    
    elements = {
        "Hα, Hβ": {"wl": [6563*u.AA, 4861*u.AA], "amp":1000*u.km/u.s, "vmax": 1300*u.km/u.s}, 
        "He II": {"wl":[4200*u.AA, 4542*u.AA, 5450*u.AA], "amp":500*u.km/u.s, "vmax": 750*u.km/u.s},
        "Fe II": {"wl":[5317*u.AA, 5169*u.AA], "amp":1000*u.km/u.s, "vmax": 1000*u.km/u.s},
        "He I":{"wl":[4388*u.AA, 4922*u.AA, 5016*u.AA], "amp":1000*u.km/u.s, "vmax": 1000*u.km/u.s},
        "Mg II": {"wl":[4481*u.AA, 7896*u.AA], "amp":300*u.km/u.s, "vmax": 400*u.km/u.s},
        "Ca II HK": {"wl":[3933*u.AA, 3968*u.AA], "amp":1500*u.km/u.s, "vmax": 1300*u.km/u.s}, 
        "Ca II": {"wl":[8498*u.AA, 8542*u.AA, 8662*u.AA], "amp":1000*u.km/u.s, "vmax": 800*u.km/u.s}, 
        "Na I": {"wl":[5890*u.AA, 5896*u.AA], "amp":800*u.km/u.s, "vmax": 750*u.km/u.s},
        #"Hδ, Hγ": {"wl":[4101*u.AA, 4340*u.AA], "amp":1000*u.km/u.s, "vmax": 1300*u.km/u.s},
        #"Si II": {"wl":[4552*u.AA, 4567*u.AA, 4574*u.AA], "amp":500*u.km/u.s, "vmax": 750*u.km/u.s},
    }
    
    #fig, axes = plt.subplots(3, 4, figsize=(10, 6))
    fig, axes = plt.subplots(2, 4, figsize=(10, 4.5))
    axes = axes.flatten()
    i=0
    
    for i, (element, line) in enumerate(elements.items()):
        amplitude=line["amp"]
        vmax=line["vmax"]
        ax=axes[i]
        if element=="Hα, Hβ":
            colors = ['darkturquoise', 'mediumvioletred', 'gold']
        else:
            colors = ['mediumvioletred', 'darkturquoise', 'gold']
        
        for j, lambda_0 in enumerate(line["wl"]):
            color = colors[j % len(colors)]
            v, norm_flux = fit_continuum(wl, flux, lambda_0, line["amp"], line["vmax"])
            ax.step(v.value, norm_flux.value, label=f"{lambda_0.value:.0f} Å", color=color, lw=0.7, where='mid')
            
            ymin = np.min(norm_flux.value)
            ymax = np.max(norm_flux.value)
            ycenter = (ymax + ymin) / 2
            yrange = (ymax - ymin) 
            if element!="Hα, Hβ":
                if element=="He I" or element=="He II" or element=="Si II":
                    ax.set_ylim(ycenter - yrange-0.1, ycenter + yrange+0.1)
                else:
                    ax.set_ylim(ycenter - yrange, ycenter + yrange)
            
        ax.text(0.95, 0.05, element, transform=ax.transAxes, fontsize=10, fontweight='bold', ha='right', va='bottom', color='black')
        legend=ax.legend(loc="upper right", fontsize=8)
        for legline in legend.get_lines():
           legline.set_linewidth(2.0)
        legend.get_frame().set_linewidth(0)
        legend.get_frame().set_facecolor('none')
        ax.xaxis.set_label_text("")

        i+=1

        for j in range(10, len(axes)):
            axes[j].set_visible(False)

    fig.supxlabel("Velocity [km/s]", fontsize=11, y=0.05)
    fig.supylabel("Normalized flux", fontsize=11, x=0.025)
    fig.text(0.07, 0.995, "YSG"+f"{os.path.basename(file)[3:-14]}", fontstyle="italic", fontweight="bold", ha='left', va='top', fontsize=10)
    #fig.suptitle("YSG"+f"{os.path.basename(file)[3:-14]}", fontsize=12)
    plt.tight_layout()
    filename = os.path.basename(file)[:-5] 
    output_path = f"{os.path.dirname(file)}/{filename}_lines.png"
    plt.subplots_adjust(hspace=0.15, wspace=0.23)
    plt.savefig(output_path, dpi=150, bbox_inches='tight')       
    plt.show()